In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from gensim.models import KeyedVectors

In [ ]:
category_mapping_path = ''
df_path = ''
output_path = ''
val_authors = ''

In [ ]:
df = pd.read_csv(category_mapping_path)

In [ ]:
df[df['article_id'] == 1169]

In [ ]:
categories = set()
for c in df.category:
    categories.add(c)

In [ ]:
train_df = pd.read_csv(df_path)

In [ ]:
len(train_df)

In [ ]:
train_df['article_id']

In [ ]:
usable_authors_test = np.load(val_authors)

In [ ]:
train_df = train_df[train_df.author_id.isin(usable_authors_test)]

In [ ]:
len(train_df)

In [ ]:
num_unique_users = train_df.author_id.nunique()

In [ ]:
num_unique_users

In [ ]:
category_to_index = { el: index for index, el in enumerate(categories)}

In [ ]:
user_to_index = {author: index for index, author in enumerate(train_df.author_id.unique())}

In [ ]:
df.index = df.article_id

In [ ]:
article_id_to_categories = df['category'].to_dict()

In [ ]:
user_matrix = np.zeros((num_unique_users, len(categories)))

In [ ]:
len(categories)

In [ ]:
error_count = 0
for index, row in tqdm(train_df.iterrows(), total=len(train_df)):
    article_keywords = article_id_to_categories.get(row.article_id)
    if article_keywords is not None:
        for keyword in article_keywords.split(','):
            user_matrix[user_to_index[row.author_id]][category_to_index[keyword]] += 1
    else:
        error_count += 1

In [ ]:
print('Articles without keywords or general Errors: ', error_count)

In [ ]:
keyed_vectors = KeyedVectors(vector_size=len(categories))

In [ ]:
keyed_vectors.add([str(author) for author in train_df.author_id.unique()], user_matrix)

In [ ]:
train_df.author_id.unique()[0]

In [ ]:
out = keyed_vectors.get_vector('258180')

In [ ]:
keyed_vectors.save_word2vec_format(output_path)